# One-body - Protein charges equilibration

In [ ]:
#Go in the working directory

try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd -q $workdir

%cd $workdir/

In [ ]:
# Import packages

import os
import json
import numpy as np
import matplotlib as mpl, matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 100%; }
    div#menubar-container     { width: 100%; }
    div#maintoolbar-container { width: 100%; }
</style>
"""))

plt.rcParams.update({'font.size':16,'legend.frameon':True,'figure.figsize':[12,8],'xtick.major.size':7,'ytick.major.size':7,'legend.labelspacing':1})

# Protein charge equilibration

In [ ]:
def mkinput():
    js = {"temperature": 298.15,
             "geometry": {"type":"sphere", "radius": r}, 
               "mcloop": {"macro": macro, "micro": micro},
               "random": {"seed": "hardware"},
               "energy": [{"nonbonded_coulomblj": 
                                 {"lennardjones": {"mixing": "LB"},
                                      "coulomb": {"type":"plain","epsr": 78.7}}}
                         ],

               "atomlist": [
                ###################
                #### TERMINALS ####
                ###################
                {"CTR": {"mw": 16.0,"sigma": 3.13,"eps": eps,"q": -1}},
                {"HCTR": {"mw": 16.0,"sigma": 3.13,"eps": eps,"q": 0}},
                {"NTR": {"mw": 14.0,"sigma": 2.99,"eps": eps,"q": 0}},
                {"HNTR": {"mw": 14.0,"sigma": 2.99,"eps": eps,"q": 1}},
                ###################
                ####Hydrophobic####
                ###################
                {"ALA": {"mw": 66.0,"sigma": 5.02,"eps": eps,"q": 0}},
                {"ILE": {"mw": 102.0,"sigma": 5.8,"eps": eps,"q": 0}},
                {"LEU": {"mw": 102.0,"sigma": 5.8,"eps": eps,"q": 0}},
                {"MET": {"mw": 122.0,"sigma": 6.16,"eps": eps,"q": 0}},
                {"PHE": {"mw": 138.0,"sigma": 6.41,"eps": eps,"q": 0}},
                {"VAL": {"mw": 90.0,"sigma": 5.56,"eps": eps,"q": 0}},
                {"PRO": {"mw": 90.0,"sigma": 5.56,"eps": eps,"q": 0}},
                {"GLY": {"mw": 54.0,"sigma": 4.69,"eps": eps,"q": 0}},
                #########################
                #### Not Hydrophobic ####
                #########################
                {"ARG": {"mw": 144.0,"sigma": 6.50,"eps": eps,"q": 0}},
                {"HARG": {"mw": 144.0,"sigma": 6.50,"eps": eps,"q": 1}},
                {"LYS": {"mw": 116.0,"sigma": 6.05,"eps": eps,"q": 0}},
                {"HLYS": {"mw": 116.0,"sigma": 6.05,"eps": eps,"q": 1}},
                {"ASP": {"mw": 110.0,"sigma": 5.94,"eps": eps,"q": -1}},
                {"HASP": {"mw": 110.0,"sigma": 5.94,"eps": eps,"q": 0}},
                {"LASP": {"mw": 110.0,"sigma": 5.94,"eps": eps,"q": 2}},
                {"GLU": {"mw": 122.0,"sigma": 6.16,"eps": eps,"q": -1}},
                {"HGLU": {"mw": 122.0,"sigma": 6.16,"eps": eps,"q": 0}},
                {"LGLU": {"mw": 122.0,"sigma": 6.16,"eps": eps,"q": 2}},
                {"GLN": {"mw": 120.0,"sigma": 6.12,"eps": eps,"q": 0}},
                {"ASN": {"mw": 108.0,"sigma": 5.91,"eps": eps,"q": 0}},
                {"HIS": {"mw": 130.0,"sigma": 6.29,"eps": eps,"q": 0}},
                {"HHIS": {"mw": 130.0,"sigma": 6.29,"eps": eps,"q": 1}},
                {"SER": {"mw": 82.0,"sigma": 5.39,"eps": eps,"q": 0}},
                {"THR": {"mw": 94.0,"sigma": 5.64,"eps": eps,"q": 0}},
                {"TYR": {"mw": 154.0, "sigma": 6.65,"eps": eps, "q": -1}},
                {"HTYR": {"mw": 154.0,"sigma": 6.65,"eps": eps,"q": 0}},
                {"CYS": {"mw": 103.0,"sigma": 5.72,"eps": eps,"q": -1}},
                {"HCYS":{"mw": 103.0,"sigma": 5.72,"eps": eps,"q": 0}},
                {"CYX": {"mw": 103.0,"sigma": 5.72,"eps": eps,"q": 0}},
                {"TRP": {"mw": 176.0,"sigma": 6.95,"eps": eps,"q": 0}},
                {"H": {"pactivity": pH, "implicit": True } }
                    ],
          
       
          "moleculelist": [{"protein": {"structure": Input, "insdir":[0,0,0], "insoffset":[0,0,0], "keepcharges":False, 'rotate': False, 'keeppos':False, 'rigid':False}},
                          ],
          
          "insertmolecules": [{"protein": {"N": 1, "inactive": False }}
                             ],
          
          "reactionlist": [ {"HCTR = CTR + H": {"pK": 3.67} },     
                            {"HASP = ASP + H": {"pK": 3.67} },
                            {"HGLU = GLU + H": {"pK": 4.25} },
                            {"HHIS = HIS + H": {"pK": 6.54} },
                            {"HNTR = NTR + H": {"pK": 8.0} }, 
                            {"HCYS = CYS + H": {"pK": 8.55} },
                            {"HTYR = TYR + H": {"pK": 9.84} }, 
                            {"HLYS = LYS + H": {"pK": 10.4} },
                            {"HARG = ARG + H": {"pK": 12.0} } ],
          
          "moves": 
                [{"rcmc": {"repeat":10} },
                ],
                 

          "analysis": [
            # Save the last charge configuration
            {"savestate": {"file": "confout_eq.pqr"} }, 

            #Total charge as a function of the number of steps
            {"reactioncoordinate": {"file": "Q_protein.dat", "nstep":1, "type": "molecule", "index": 0, "property": "Q", 'nskip': nskip}},
           
            ]}  

    with open('input.json', 'w') as f:
        json.dump(js, f, indent=4)


###################################################################
pHs = {'pH2': { 'pH': [2],  'input':workdir+'/../../PPI40_cg.pqr'},
       'pH5': { 'pH': [5],  'input':workdir+'/../../PPI40_cg.pqr'} 
      }
###################################################################

#LJ parameters                                                                                                                                       
eps = 0.12395 # epsilon for not hydrophobic in kJ/mol equal to 0.05 kT  

# Radius
r =30 
                                                                                                         
# Steps                                                                                                         
micro = 1000
macro = 10
nskip = 0


###############################################################################################################################

for name, prop in pHs.items():
    %mkdir $name
    %cd -q $name
    for pH in prop ['pH']:
        %mkdir $pH
        %cd -q $pH

        Input= prop['input'] 
        mkinput()

        exists = os.path.isfile('state.json')
        #if exists:
            #!export OMP_NUM_THREADS=1;/Users/marco/faunus/faunus -i input.json -s state.json -v 6 #> out
        #    !sbatch ../../../submit_state.sh   
        #else:
            #!export OMP_NUM_THREADS=1;/Users/marco/faunus/faunus -i input.json  -v 6 #> out
        #    !sbatch ../../../submit.sh
                
        %cd -q ../
    %cd -q ../

In [ ]:
!squeue -u marpoli
#!scancel 5700667

## Check charge equilibration

In [ ]:
################################################################
pHs = {'pH2': { 'pH': [2],  'input':workdir+'/../PPI40_cg.pqr'},
       'pH5': { 'pH': [5],  'input':workdir+'/../PPI40_cg.pqr'} 
      }
################################################################


for name, prop in pHs.items():
    for pH in prop ['pH']:
        path = '/'+str(name)+'/'+str(pH)+'/'
        step, Q = np.loadtxt(workdir+path+'Q_protein.dat',unpack=True, usecols=(0,1))
        
        plt.plot(step, Q, 'x-', label=str(r))
        plt.xlabel('Steps')
        plt.ylabel('Q (e)')
        print('Last total charge is: ', Q[-1], 'e')

# Create inputs for production runs

In [ ]:
# Load equilibrated charges
r2_pH2, r8_pH2  = np.loadtxt('pH2/2/confout_eq.pqr', skiprows=1, unpack=True, usecols=(2, 8), dtype=str) # remove the 'END' before to run it
r2_pH5, r8_pH5  = np.loadtxt('pH5/5/confout_eq.pqr', skiprows=1, unpack=True, usecols=(2, 8), dtype=str) # remove the 'END' before to run it

# Load intial file 
r0, r1, r2, r3, r4, r5, r6, r7, r8, r9  = np.loadtxt('../../PPI40_cg.pqr', skiprows=1, unpack=True, usecols=(0,1,2,3,4,5,6,7,8,9), dtype=str)

In [ ]:
# pH2
with open('Q_eq_pH2.pqr','w') as f: 
    f.write("42\n")
    writer = csv.writer(f, delimiter="\t",lineterminator='\n') 
    writer.writerows(zip(r0, r1, r2_pH2, r2_pH2, r4, r5, r6, r7, r8_pH2, r9))
    f.write('END\n')

In [ ]:
# pH5
with open('Q_eq_pH5.pqr','w') as f: 
    f.write("42\n")
    writer = csv.writer(f, delimiter="\t",lineterminator='\n') 
    writer.writerows(zip(r0, r1, r2_pH5, r2_pH5, r4, r5, r6, r7, r8_pH5, r9))
    f.write('END\n')